# Let's go!

## Imports

In [ ]:
import normflows as nf
import matplotlib.pyplot as plt
import torch
import numpy as np

from fab import FABModel, HamiltonianMonteCarlo, Metropolis
from fab.utils.logging import ListLogger
from fab.utils.plotting import plot_history, plot_contours, plot_marginal_pair
from fab.target_distributions.gmm import GMM
from fab.utils.prioritised_replay_buffer import PrioritisedReplayBuffer
from fab import Trainer, PrioritisedBufferTrainer
from fab.utils.plotting import plot_contours, plot_marginal_pair
from tqdm import tqdm,trange
import scipy.stats.qmc as qmc
from experiments.make_flow import make_wrapped_normflow_realnvp
import pandas as pd

## Setup Target distribution

In [ ]:
dim = 2
n_mixes = 40
loc_scaling = 40.0  # scale of the problem (changes how far apart the modes of each Guassian component will be)
log_var_scaling = 1.0 # variance of each Gaussian
seed = 1

In [ ]:
torch.manual_seed(0)  # seed of 0 for GMM problem
target = GMM(dim=dim, n_mixes=n_mixes,
              loc_scaling=loc_scaling, log_var_scaling=log_var_scaling,
              use_gpu=True, true_expectation_estimation_n_samples=int(1e5))

In [ ]:
# plot target
target.to("cpu")
fig, ax = plt.subplots()
plotting_bounds = (-loc_scaling * 1.4, loc_scaling * 1.4)
plot_contours(target.log_prob, bounds=plotting_bounds, n_contour_levels=80, ax=ax, grid_width_n_points=200)
target.to("cuda")

## Create FAB model

In [ ]:
# hyper-parameters

# Flow
n_flow_layers = 15
layer_nodes_per_dim = 40
lr = 1e-4
max_gradient_norm = 100.0
batch_size = 128
n_iterations = 4000
n_eval = 10
eval_batch_size = batch_size * 10
n_plots = 10 # number of plots shows throughout tranining
use_64_bit = True
alpha = 2.0

# AIS
# By default we use a simple metropolis mcmc transition with a fixed step size.
# Can switch this to 'hmc' to improve training efficiency. 
transition_operator_type = "metropolis" 
n_intermediate_distributions = 1
metropolis_step_size = 5.0

# buffer config
n_batches_buffer_sampling = 4
maximum_buffer_length = batch_size * n_batches_buffer_sampling * 100
min_buffer_length = batch_size * n_batches_buffer_sampling * 10

# target p^\alpha q^{a-\alpha} as target for AIS. 
min_is_target = True
p_target = not min_is_target # Whether to use p as the target. 

In [ ]:
if use_64_bit:
    torch.set_default_dtype(torch.float64)
    target = target.double()
    print(f"running with 64 bit")

### Setup flow

In [ ]:
flow = make_wrapped_normflow_realnvp(dim, n_flow_layers=n_flow_layers, 
                                 layer_nodes_per_dim=layer_nodes_per_dim,
                                act_norm = False)

### Setup Transition operator

In [ ]:
if transition_operator_type == "hmc":
    # very lightweight HMC.
    transition_operator = HamiltonianMonteCarlo(
            n_ais_intermediate_distributions=n_intermediate_distributions,
            dim=dim,
            base_log_prob=flow.log_prob,
            target_log_prob=target.log_prob,
            alpha=alpha,
            p_target=p_target,
        n_outer=1,
        epsilon=1.0, L=5)
elif transition_operator_type == "metropolis":
    transition_operator = Metropolis(            
        n_ais_intermediate_distributions=n_intermediate_distributions,
        dim=dim,
        base_log_prob=flow.log_prob,
        target_log_prob=target.log_prob,
        p_target=p_target,
        alpha=alpha,
        n_updates=1,
        adjust_step_size=False,
        max_step_size=metropolis_step_size, # the same for all metropolis steps 
        min_step_size=metropolis_step_size,
        eval_mode=False,
                                  )
else:
    raise NotImplementedError

### Setup FAB model with prioritised replay buffer

In [ ]:
# use GPU if available
if torch.cuda.is_available():
    flow.cuda()
    transition_operator.cuda()
    target.to("cuda")
    print(f"Running with GPU")

In [ ]:
fab_model = FABModel(flow=flow,
                     target_distribution=target,
                     n_intermediate_distributions=n_intermediate_distributions,
                     transition_operator=transition_operator,
                     alpha=alpha)
optimizer = torch.optim.Adam(flow.parameters(), lr=lr)
logger = ListLogger(save=False) # save training history

In [ ]:
# Setup buffer.
def initial_sampler():
  # fill replay buffer using initialised model and AIS.
    point, log_w = fab_model.annealed_importance_sampler.sample_and_log_weights(
            batch_size, logging=False)
    return point.x, log_w, point.log_q
buffer = PrioritisedReplayBuffer(dim=dim, max_length=maximum_buffer_length,
                      min_sample_length=min_buffer_length,
                      initial_sampler=initial_sampler)

In [ ]:
def plot(fab_model, n_samples = 128):
    fig, axs = plt.subplots(1, 3, figsize=(12, 4))
    target.to("cpu")
    plot_contours(target.log_prob, bounds=plotting_bounds, ax=axs[0], n_contour_levels=50, grid_width_n_points=200)
    plot_contours(target.log_prob, bounds=plotting_bounds, ax=axs[1], n_contour_levels=50, grid_width_n_points=200)
    plot_contours(target.log_prob, bounds=plotting_bounds, ax=axs[2], n_contour_levels=50, grid_width_n_points=200)
    target.to("cuda")

    # plot flow samples
    samples_flow = fab_model.flow.sample((n_samples,)).detach()
    plot_marginal_pair(samples_flow, ax=axs[0], bounds=plotting_bounds)


    # plot ais samples
    samples_ais = fab_model.annealed_importance_sampler.sample_and_log_weights(n_samples,
                                                                               logging=False)[0].x
    plot_marginal_pair(samples_ais, ax=axs[1], bounds=plotting_bounds)
    
    # plot buffer samples
    samples_buffer = buffer.sample(n_samples)[0].detach()
    plot_marginal_pair(samples_buffer, ax=axs[2], bounds=plotting_bounds)
    
    axs[0].set_title("flow samples")
    axs[1].set_title("ais samples")
    axs[2].set_title("buffer samples")
    plt.show()
    return [fig]

In [ ]:
plot(fab_model) # Visualise model during initialisation.

In [ ]:
# Setup trainer.
trainer = PrioritisedBufferTrainer(model=fab_model, optimizer=optimizer, 
              logger=logger, plot=plot,
              buffer=buffer, 
              n_batches_buffer_sampling=n_batches_buffer_sampling,
              max_gradient_norm=max_gradient_norm,
              alpha=alpha,
              w_adjust_max_clip=None)

## Train model

This problem is quite challenging for training, as the flow has a very poor initialisation, and therefore often places extremely small probability on samples in new modes.

This causes some **numerical instability**: There are lots of NaN errors throughout training, due to (1) numerical sinstability in the flow itself causing the flow to generate NaN samples or samples with large values that have NaN log prob under the target, as well as (2) sometimes AIS finds regions in the target with negligible mass under the flow.  However, these numerical instabilities do not prevent training from suceeding. 

In [ ]:
# Now run!
trainer.run(n_iterations=n_iterations, batch_size=batch_size, n_plot=n_plots, \
            n_eval=n_eval, eval_batch_size=eval_batch_size, save=False)  # note that the progress bar during training prints ESS w.r.t p^2/q. 

In the below plot of samples from the flow vs the target contours, and with the test set log prob throughout training, we see that the flow covers the target distribution quite well. It may be trained further to obtain even better results. 

In [ ]:
# Test set probability using samples from the target distribution.
eval_iters = np.linspace(0, n_iterations, n_eval)
plt.plot(logger.history['flow_test_set_mean_log_prob_p_target'])
plt.ylabel("mean test set log prob")
plt.xlabel("eval iteration")
plt.yscale("symlog")

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(5, 5))
target.to("cpu")
plot_contours(target.log_prob, bounds=plotting_bounds, ax=axs, n_contour_levels=50, grid_width_n_points=200)
target.to("cuda")

n_samples = 1000
samples_flow = fab_model.flow.sample((n_samples,)).detach()
plot_marginal_pair(samples_flow, ax=axs, bounds=plotting_bounds)

In [ ]:
import scipy.stats.qmc as qmc
fig, axs = plt.subplots(1, 1, figsize=(5, 5))
target.to("cpu")
plot_contours(target.log_prob, bounds=plotting_bounds, ax=axs, n_contour_levels=50, grid_width_n_points=200)
target.to("cuda")
gauss_flow = fab_model.flow._nf_model.q0
mean = gauss_flow.loc.data.cpu().numpy()[0]
scale = np.diag(np.exp(gauss_flow.log_scale.data.cpu().numpy())[0])

cov = None
# sample_qmc = qmc.MultivariateNormalQMC(mean = mean,cov = scale).random(1024)
# sample_qmc_cuda = torch.tensor(sample_qmc)
# samples_flow_qmc = fab_model.flow._nf_model.forward(sample_qmc_cuda).detach()
samples, log_q = fab_model.flow.sample_and_log_prob((1024, ))

plot_marginal_pair(samples, ax=axs, bounds=plotting_bounds)

In [ ]:
def evaluate_qmc(funs,n_repeats=10): 
    """Evaluate by estimating quadratic function. If `path_to_model==target` then this is done
    using samples from the target"""
    biases = []
    biases_unweighted = []
    num_samples = 2**16
    mean_list = []
    list_dict = []
    for i in trange(n_repeats):
        model = fab_model
        samples, log_q = model.flow.sample_and_log_prob_qmc((num_samples, ),seed = i+1000)
        log_w = target.log_prob(samples) - log_q
        valid_indices = ~torch.isinf(log_w) & ~torch.isnan(log_w)
        samples, log_w = samples[valid_indices], log_w[valid_indices]
        valid_indices_unweighted = ~ (torch.softmax(log_w, axis=0) == 0)
        samples_unweighted = samples[valid_indices_unweighted]
        log_w_unweighted = torch.ones_like(log_w[valid_indices_unweighted])
        normed_bias = target.evaluate_expectation(samples, log_w).detach().cpu()
        normed_bias_unweighted = target.evaluate_expectation(samples_unweighted,
                                                             log_w_unweighted).detach().cpu()
        biases.append(normed_bias)
        biases_unweighted.append(normed_bias_unweighted)
        for fun,name in funs:
            mean = torch.mean(fun(samples)*torch.exp(log_w))
            list_dict.append({"value":float(mean.detach().cpu().numpy()), "function": name,"iter": i,"method": "qmc"})
        mean_list.append(mean.detach().cpu().numpy())
    info = {"bias": np.mean(np.abs(biases)),
            "std": np.std(biases),
            "bias_unweighted": np.mean(np.abs(biases_unweighted)),
            "df": list_dict}
    return info
funs = [(lambda x :x[:,0],"x[0]"), (lambda x :x[:,1],"x[1]"), (lambda x :x[:,0]**2,"x[0]**2"), (lambda x :x[:,1]**6,"x[1]**6"), (lambda x :x[:,0]*x[:,1],"cov"),(lambda x: torch.sin(x[:,0])*torch.cos(-x[:,1]/10),"sincos"), (lambda x: (x[:,0]>30)*1.0,"indi")]
u0 = evaluate_qmc(funs,200)

In [ ]:
def evaluate(funs,n_repeats=10): 
    """Evaluate by estimating quadratic function. If `path_to_model==target` then this is done
    using samples from the target"""
    biases = []
    biases_unweighted = []
    num_samples = 2**16
    mean_list = []
    list_dict = []
    for i in trange(n_repeats):
        model = fab_model
        samples, log_q = model.flow.sample_and_log_prob((num_samples, ))
        log_w = target.log_prob(samples) - log_q
        valid_indices = ~torch.isinf(log_w) & ~torch.isnan(log_w)
        samples, log_w = samples[valid_indices], log_w[valid_indices]
        valid_indices_unweighted = ~ (torch.softmax(log_w, axis=0) == 0)
        samples_unweighted = samples[valid_indices_unweighted]
        log_w_unweighted = torch.ones_like(log_w[valid_indices_unweighted])
        normed_bias = target.evaluate_expectation(samples, log_w).detach().cpu()
        normed_bias_unweighted = target.evaluate_expectation(samples_unweighted,
                                                             log_w_unweighted).detach().cpu()
        biases.append(normed_bias)
        biases_unweighted.append(normed_bias_unweighted)

        for fun,name in funs:
            mean = torch.mean(fun(samples)*torch.exp(log_w))
            list_dict.append({"value":float(mean.detach().cpu().numpy()), "function": name,"iter": i,"method": "mc"})
        mean_list.append(mean.detach().cpu().numpy())
    info = {"bias": np.mean(np.abs(biases)),
            "std": np.std(biases),
            "bias_unweighted": np.mean(np.abs(biases_unweighted)),
            "df": list_dict}
    return info

u1 = evaluate(funs, 200)


In [ ]:
import pandas as pd
import seaborn as sns
df = pd.DataFrame(u1["df"] + u0["df"])
df["newcol"] = df.groupby(["function"])["value"].transform(lambda x : (x- x.mean())/ x.std())
sns.boxenplot(x="function", y="newcol", hue = "method",data=df)
plt.ylabel("normalized estimate")
plt.title("Normalized estimate of different functions")
plt.savefig("Latex/normalized_estimate.pdf")

In [ ]:
u = df[df["method"] == "mc"].groupby(["function"])["value"].std() / df[df.method == "qmc"].groupby(["function"])["value"].std()

In [ ]:
print(u.to_frame().T.to_latex(index=False, float_format="%.3f"))